# Webscrape Pokedex Data

##### Project Set Up:
1. Web-scrape data (www.pokemondb.net) and organize into a table
2. Web-scrape images of each pokemon
3. Utilize data to deploy app

In [ ]:
# Retrieve data on the Pokemon
from selenium import webdriver  # webdriver is what is driving the action
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys  # Gives access to keys such as enter, space, escape.
from selenium.webdriver.common.by import By  # These three imports are required for wait.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def pokemon_names():
    global names
    global links
    names = []
    links = []
    PATH = 'XXXXXXXXXXXXXX'
    driver = webdriver.Chrome(PATH)
    region = 'https://pokemondb.net/pokedex/game/firered-leafgreen'     # Kanto Region
    driver.get(region)
    global pokedex
    pokedex = driver.find_elements(By.CLASS_NAME,'ent-name')
    for i in range(len(pokedex)):
        links.append(pokedex[i].get_attribute('href'))

    for i in range(len(pokedex)):
       names.append(pokedex[i].text)
pokemon_names()
# print(names)
# print(links)

In [ ]:
def find_stats():
    global stats
    from selenium.common.exceptions import NoSuchElementException
    stats = {'Name':[],'Type1':[],'Type2':[],'Species':[],'Height':[],'Weight':[],'Entry':[],'First Ability':[],'Second Ability':[]}
    PATH = 'XXXXXXXXXXXXXX'
    driver = webdriver.Chrome(PATH)
    for i in range(len(pokedex)):
        pokemon = links[i]
        driver.get(pokemon)
        types = driver.find_elements(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[2]/td/a')
        species = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[3]/td')
        height = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[4]/td')
        weight = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[5]/td')
        entry = driver.find_element(By.CLASS_NAME,'cell-med-text')
        ability1 = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[6]/td/span/a')
        try:
            ability2 = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[2]/table/tbody/tr[6]/td/small/a')
            ability2 = ability2.text
        except NoSuchElementException:
            ability2 = 'None'

        try:
            type1 = str(types[0].text)
            type2 = str(types[1].text)
            time.sleep(2)
        except IndexError:
            type1 = types[0].text
            type2 = ''

        stats['Name'].append(names[i])
        stats['Type1'].append(type1)
        stats['Type2'].append(type2)
        stats['Species'].append(species.text)
        stats['Height'].append(height.text)
        stats['Weight'].append(weight.text)
        stats['Entry'].append(entry.text)
        stats['First Ability'].append(ability1.text)
        stats['Second Ability'].append(ability2)
    import pandas as pd
    stats = pd.DataFrame(stats, columns=['Name','Type1','Type2','Species','Height','Weight','Entry','First Ability','Second Ability'])
    print(stats)
find_stats()

In [ ]:
# Create a function to download all pokemon images
import requests
import shutil

names
links

def get_image():
    PATH = 'XXXXXXXXXXXXXX'
    driver = webdriver.Chrome(PATH)
    for i in range(len(pokedex)):
        driver.get(links[i])
        img = driver.find_element(By.XPATH,'/html/body/main/div[2]/div[2]/div/div[1]/div[1]/p[1]/a/picture/img')
        src = img.get_attribute('src')
        url = src
        response = requests.get(url,stream=True)
        with open(f'{names[i]}.jpg','wb') as out_file:
            shutil.copyfileobj(response.raw,out_file)
        del response
get_image()

In [ ]:
# Export Pokemon stats as an excel file.

import pandas as pd
with pd.ExcelWriter('XXXXXXXXXXXXXX/Pokedex.xlsx') as writer:
    stats.to_excel(writer,sheet_name='Kanto',index=False)
        